In [1]:
import scanpy as sc
import numpy as np
import csv
import pickle
import matplotlib
import math
from itertools import combinations
import itertools
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite
from cdlib import algorithms
from cdlib import NodeClustering
import altair as alt
import sys
import os
sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
if True:  # In order to bypass isort when saving
    import altairThemes

Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'wurlitzer', 'infomap', 'graph_tool'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [6]:
# Read in the whole log odds table
whole_log_odds_df = pd.read_csv('/Users/victoriagao/local_docs/NEST/output/LR_log_odds/BIG_log_odds.csv')

output_folder = "/Users/victoriagao/local_docs/NEST/output/gsea_results/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [7]:
whole_log_odds_df

,L-R Pair,region1 VS 2,region1 VS 3,region1 VS 4,region1 VS 5,region1 VS 6,region1 VS 7,region1 VS 8,region1 VS 9,region1 VS 10,...,region17 VS 18,region17 VS 19,region17 VS 20,region17 VS 21,region18 VS 19,region18 VS 20,region18 VS 21,region19 VS 20,region19 VS 21,region20 VS 21
0,LGALS3-NPTN,-1.838279,0.350977,0.492476,0.860201,inf,-0.287682,inf,inf,inf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FN1-RPSA,NaN,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PTPRF-RACK1,inf,-1.210730,inf,-0.416894,inf,inf,inf,inf,inf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INS-HLA,NaN,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TIMP1-LRP1,NaN,-inf,NaN,-inf,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,BMP4-BMPR2,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,BMP2-ENG,NaN,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,HMGB1-AGTRAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,TGFB1-SDC2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
###### This only plots the non-infinitive log odds values
# Separate negative and positive values
negative_values = log_odds_df[(log_odds_df['log_odds_ratio'] < 0) & (log_odds_df['log_odds_ratio'] != -float('inf'))]

# negative_values = log_odds_df[(log_odds_df['log_odds_ratio'] < 0) | (log_odds_df['log_odds_ratio'] == -float('inf'))]
positive_values = log_odds_df[(log_odds_df['log_odds_ratio'] > 0) & (log_odds_df['log_odds_ratio'] != float('inf'))]

# Create tick mark list
# tick_list = pd.concat([negative_values["L-R Pair"],positive_values["L-R Pair"]])

# Create a bar plot
fig, ax = plt.subplots(figsize=(35, 11)) 
# fig, ax = plt.subplots()
ax.bar(negative_values['L-R Pair'], negative_values['log_odds_ratio'], color='green', label='Negative')
ax.bar(positive_values['L-R Pair'], positive_values['log_odds_ratio'], color='red', label='Positive')

# Add labels, title, and legend
ax.set_xlabel('L-R Pair')
ax.set_ylabel('log_odds_ratio')
ax.set_title('Bar Plot of log_odds_ratio')
ax.legend()

# Rotate x-axis labels for better readability
# plt.xticks(tick_list,rotation=45, ha='right')
# plt.xticks(log_odds_df["L-R Pair"],rotation=45, ha='right')
plt.xticks(rotation=45, ha='right')
# plt.xticks(positive_values['L-R Pair'],rotation=45, ha='right')

In [ ]:

# Get the L-R pair information from the first column of master_log_odds_df
LR_pairs_info = whole_log_odds_df.iloc[:, 0]

# Iterate through each column of master_log_odds_df, excluding the first column
for log_odds_values in whole_log_odds_df.iloc[:, 1:].items():
    
    negative_values = log_odds_df[(log_odds_df['log_odds_ratio'] < 0) & (log_odds_df['log_odds_ratio'] != -float('inf'))]

# negative_values = log_odds_df[(log_odds_df['log_odds_ratio'] < 0) | (log_odds_df['log_odds_ratio'] == -float('inf'))]
    positive_values = log_odds_df[(log_odds_df['log_odds_ratio'] > 0) & (log_odds_df['log_odds_ratio'] != float('inf'))]

    # Create tick mark list
    # tick_list = pd.concat([negative_values["L-R Pair"],positive_values["L-R Pair"]])

    # Create a bar plot
    fig, ax = plt.subplots(figsize=(35, 11)) 
    # fig, ax = plt.subplots()
    ax.bar(negative_values['L-R Pair'], negative_values['log_odds_ratio'], color='green', label='Negative')
    ax.bar(positive_values['L-R Pair'], positive_values['log_odds_ratio'], color='red', label='Positive')

    # Add labels, title, and legend
    ax.set_xlabel('L-R Pair')
    ax.set_ylabel('log_odds_ratio')
    ax.set_title('Bar Plot of log_odds_ratio')
    ax.legend()

    # Rotate x-axis labels for better readability
    # plt.xticks(tick_list,rotation=45, ha='right')
    # plt.xticks(log_odds_df["L-R Pair"],rotation=45, ha='right')
    plt.xticks(rotation=45, ha='right')

    


In [18]:

# Iterate through each column of the master_log_odds_df starting from the second column
for column_name, log_odds_values in whole_log_odds_df.iloc[:, 1:].items():
    # Select data for the current column
    # log_odds_df = whole_log_odds_df[[column]].dropna()
    log_odds_df = pd.DataFrame({column_name: log_odds_values})
    column_name = pd.DataFrame({column_name: column_name})

    # Concatenate the new DataFrame with the results DataFrame horizontally
    results_df = pd.concat([column_name, log_odds_df], axis=1)

    # Separate positive and negative values
    # negative_values = log_odds_values[(log_odds_values < 0) & (log_odds_values != -float('inf'))].sort_values(ascending=False)
    # positive_values = log_odds_values[(log_odds_values > 0) & (log_odds_values != float('inf'))].sort_values(ascending=False)
    negative_values = log_odds_df[(log_odds_df['log_odds_ratio'] < 0) & (log_odds_df['log_odds_ratio'] != -float('inf'))]
    # negative_values = log_odds_df[(log_odds_df['log_odds_ratio'] < 0) | (log_odds_df['log_odds_ratio'] == -float('inf'))]
    positive_values = log_odds_df[(log_odds_df['log_odds_ratio'] > 0) & (log_odds_df['log_odds_ratio'] != float('inf'))]

    # Create a bar plot
    fig, ax = plt.subplots(figsize=(35, 11))
    ax.bar(negative_values.index, negative_values, color='blue', label='Negative')
    ax.bar(positive_values.index, positive_values, color='red', label='Positive')

    # Add labels, title, and legend
    ax.set_xlabel('L-R Pair')
    ax.set_ylabel('log_odds_ratio')
    ax.set_title(f'Bar Plot of {column}')
    ax.legend()

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, ha='right')

    # Save the plot to a file (optional)
    # You can save the plot to a file if needed.
    # Replace 'output_folder' and 'file_name' with your desired file path and name.
    # file_name = f"{column}_bar_plot.png"
    # plt.savefig(os.path.join(output_folder, file_name))

    # Show the plot (optional)
    # Uncomment the following line if you want to display the plot interactively.
    # plt.show()

# To close all plots after generating them (optional)
# plt.close('all')


ValueError: If using all scalar values, you must pass an index